IMPORTAÇÃO

In [11]:
from config_path import *
from fuction import Funcao
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pyodbc
import glob
import os
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

CARGA

In [12]:
DB_ACUMULADO = r'C:\Users\wesley.oliveira\WS_OLIVEIRA\BASE_DADOS\ACUMULADOS\db_cheio_vazio.accdb'
DRIVER = '{Microsoft Access Driver (*.mdb, *.accdb)}'
NOME_TABELA = 'ACUMULADO'

try:
    conn = pyodbc.connect(
        f"DRIVER={DRIVER};"
        f"DBQ={DB_ACUMULADO};"
    )

    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn}")
    print("Conectado!")
except Exception as e:
    print(f"Falha na conexão com o Access: {e}")
    
cursor = conn.cursor()

for table_info in cursor.tables(tableType='table'):
        nome_tabela = table_info[2]
        tipo = table_info[3] 
        print(f"Nome: {nome_tabela} (Tipo: {tipo})")

Conectado!
Nome: ACUMULADO (Tipo: TABLE)


TRATAMENTO

In [13]:
query_consulta = f"SELECT * FROM {NOME_TABELA}"

cursor.execute(query_consulta)

dados = cursor.fetchall()
colunas = [column[0] for column in cursor.description]
df_bd = pd.DataFrame.from_records(dados, columns=colunas)
display(df_bd)


,Código,END,COD,DESC,RUA,PREDIO,NIVEL,APTO,RETORNO,DATA_RELATORIO,NOME_RELATORIO


In [14]:
pasta_files = glob.glob(os.path.join(directory.dir_cheio_vazio, "*xls*"))

list_processado = []
list_erros = {}

for file in pasta_files:
    NAME_FILE = os.path.basename(file)
    try:
        df = pd.read_excel(file, header= 2)
        df_filtrado = df.loc[df['Retorno'].isin(['CHEIO', 'VAZIO'])]
        df_filtrado = df_filtrado[['END', 'COD', 'DESCRIÇÃO', 'RUA', 'PREDIO', 'NIVEL', 'APTO', 'Retorno', 'data_relatorio']]
        df_filtrado['NAME_FILE'] = NAME_FILE
        list_processado.append(df_filtrado)
    except Exception as e:
        list_erros['ETAPA_CONSOLIDADO'] = str(NAME_FILE), str(e)

print(f"erros: {list_erros}")
df_consolidado = pd.concat(list_processado, axis= 'index')

col_int = ['END', 'COD','RUA','PREDIO','NIVEL','APTO']
for col in col_int:
    try:
        df_consolidado[col] = df_consolidado[col].fillna(0).astype(int)
    except Exception as e:
        list_erros.clear()
        list_erros[col] = str(e)



erros: {'ETAPA_CONSOLIDADO': ('VAZIO-19-11.xlsx', '"[\'data_relatorio\'] not in index"')}


CONSULTA

CARGA

In [ ]:
"""
    CONECÇÃO COM OS EXCEL
"""
# with pd.ExcelWriter(ipynb.retorno) as fim:
#     df_sandalia.to_excel(fim, index= False, sheet_name= "RETORNO")

"""
   CONECÇÃO COM O ACCESS
 """

# try:
#     df_exemplo.to_sql(
#         name=NOME_TABELA,
#         con=engine,
#         if_exists='append', # 'replace' (remover e recriar) ou 'append' (adicionar)
#         index=False           # Não salvar o índice do DataFrame como coluna
#     )
#     print(f"✅ DataFrame salvo com sucesso na tabela '{NOME_TABELA}' no Access.")
# except Exception as e:
#     print(f"❌ Ocorreu um erro ao salvar os dados: {e}")